In [3]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
import os
from dotenv import load_dotenv

# Configuration Langfuse pour le monitoring
from langfuse.openai import openai
from langfuse import Langfuse

load_dotenv()

# Configuration des clés Langfuse
langfuse = Langfuse(
    public_key=os.getenv('LANGFUSE_PUBLIC_KEY'),
    secret_key=os.getenv('LANGFUSE_SECRET_KEY'),
    host=os.getenv('LANGFUSE_HOST', 'https://cloud.langfuse.com')
)

class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)
        
        # Remplacer le client OpenAI par la version instrumentée Langfuse
        import openai as openai_original
        if hasattr(self, '_client'):
            # Sauvegarder la clé API
            api_key = getattr(self._client, 'api_key', config.get('api_key'))
            # Remplacer par le client Langfuse
            self._client = openai.OpenAI(api_key=api_key)
    
    def submit_prompt(self, prompt, **kwargs):
        """Override pour ajouter le tracking Langfuse"""
        with langfuse.trace(
            name="vanna-query",
            input={"prompt": prompt, "kwargs": kwargs},
            user_id=kwargs.get('user_id', 'anonymous'),
            session_id=kwargs.get('session_id', 'default'),
            tags=["vanna", "sql-generation"]
        ) as trace:
            try:
                result = super().submit_prompt(prompt, **kwargs)
                trace.update(output={"result": result})
                return result
            except Exception as e:
                trace.update(output={"error": str(e)})
                raise

vn = MyVanna(config={'api_key':os.getenv('OPENAI_API_KEY'), 'model': 'gpt-4o-mini'})
# vn.remove_training_data()

vn.connect_to_postgres(host=os.getenv('POSTGRES_HOST'), dbname=os.getenv('POSTGRES_DB'), user=os.getenv('POSTGRES_USER'), password=os.getenv('POSTGRES_PASSWORD'), port=os.getenv('POSTGRES_PORT'))

In [ ]:
# Ajout du monitoring personnalisé pour l'application Flask
import functools
from flask import request

def track_flask_requests(app_instance):
    """Décorateur pour tracker les requêtes Flask avec Langfuse"""
    original_ask = app_instance.vn.ask
    
    @functools.wraps(original_ask)
    def tracked_ask(question, **kwargs):
        # Récupérer les informations de la requête
        session_id = request.session.get('session_id', 'flask-session')
        user_id = request.remote_addr  # IP comme ID utilisateur basique
        
        with langfuse.trace(
            name="vanna-flask-ask",
            input={"question": question, "kwargs": kwargs},
            user_id=user_id,
            session_id=session_id,
            tags=["vanna", "flask", "text-to-sql"],
            metadata={
                "user_agent": request.headers.get('User-Agent'),
                "method": request.method,
                "endpoint": request.endpoint
            }
        ) as trace:
            try:
                # Mesurer la latence
                import time
                start_time = time.time()
                
                result = original_ask(question, **kwargs)
                
                end_time = time.time()
                latency_ms = (end_time - start_time) * 1000
                
                trace.update(
                    output={"result": result},
                    metadata={"latency_ms": latency_ms}
                )
                
                # Enregistrer les métriques de coût si disponibles
                if hasattr(result, 'usage'):
                    trace.update(
                        usage={
                            "input_tokens": getattr(result.usage, 'prompt_tokens', 0),
                            "output_tokens": getattr(result.usage, 'completion_tokens', 0),
                            "total_tokens": getattr(result.usage, 'total_tokens', 0)
                        }
                    )
                
                return result
            except Exception as e:
                trace.update(
                    output={"error": str(e)},
                    level="ERROR"
                )
                raise
    
    app_instance.vn.ask = tracked_ask
    return app_instance

from vanna.flask import VannaFlaskApp

# Créer l'application Flask avec monitoring
flask_app = VannaFlaskApp(vn, allow_llm_to_see_data=True)

# Appliquer le tracking Langfuse
flask_app = track_flask_requests(flask_app)

print("🔍 Langfuse monitoring activé!")
print(f"📊 Dashboard: {os.getenv('LANGFUSE_HOST', 'https://cloud.langfuse.com')}")
print("🚀 Toutes les requêtes seront trackées automatiquement...")

# Lancer l'application
flask_app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
Using model gpt-4o-mini for 4746.25 tokens (approx)
Using model gpt-4o-mini for 102.75 tokens (approx)
